# 第五章 检索(Retrieval)

 - [一、向量数据库检索](#一、向量数据库检索)
     - [1.1 相似性搜索](#1.1-相似性搜索)
     - [1.2 解决多样性：最大边际相关性(MMR)](#1.2-解决多样性：最大边际相关性(MMR))
     - [1.3 解决特殊性：使用元数据](#1.3-解决特殊性：使用元数据)
     - [1.4 解决特殊性：在元数据中使用自查询检索器](#1.4-解决特殊性：在元数据中使用自查询检索器)
     - [1.5 其他技巧：压缩](#1.5-其他技巧：压缩)
 - [二、结合各种技术](#二、结合各种技术)
 - [三、其他类型的检索](#三、其他类型的检索)



检索是我们的检索增强生成(RAG)流程的核心。

让我们获得前面课程存储的向量数据库(`VectorDB`)。

## 一、向量数据库检索

在当前文件夹下新建`.env`文件，内容为`OPENAI_API_KEY = "sk-..."`

本章节需要使用`lark`包，运行以下命令安装

In [2]:
# !pip install -Uq lark

In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']


d:\Anaconda3\envs\LLM\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### 1.1 相似性搜索

将上节课所保存的向量数据库(`VectorDB`)加载进来

In [16]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/cs229_lectures/'
persist_directory_chinese = 'docs/chroma/matplotlib/'


embedding_model = OpenAIEmbeddings(
    # model_name = "gpt-4"
)
vector_db = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model
)
print(vector_db._collection.count())

vector_db_chinese = Chroma(
    persist_directory=persist_directory_chinese,
    embedding_function=embedding_model
)
print(vector_db_chinese._collection.count())

836
74


d:\Anaconda3\envs\LLM\lib\site-packages\langchain\embeddings\openai.py:214: UserWarning: WARNING! model_name is not default parameter.
                    model_name was transferred to model_kwargs.
                    Please confirm that model_name is what you intended.
  warnings.warn(


让我们现在来看看最大边际相关性的例子。因此，我们将从下面示例中加载有关蘑菇的信息。

让我们现在运行它与MMR。让我们传入k等于2。我们仍然希望返回两个文档，但让我们设置获取k等于3，其中我们最初获取所有三个文档。我们现在可以看到，我们检索的文档中返回了有毒的信息。

In [17]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

texts_chinese = [
    """毒鹅膏菌（Amanita phalloides）具有大型且引人注目的地上（epigeous）子实体（basidiocarp）""",
    """一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。""",
    """A. phalloides，又名死亡帽，是已知所有蘑菇中最有毒的一种。""",
]

对于这个例子，我们将创建一个小数据库，我们可以作为一个示例来使用。

对于示例提出问题,并运行一个相似性搜索，设置k=2，只返回两个最相关的文档。

我们可以看到，没有提到它是有毒的事实。


之后，让我们运行最大边际相关性(MMR)。

设置k=2，因为我们仍然希望返回两个文档。设置fetch_k=3，fetch_k是我们最初获取的所有文档(3个)。


In [18]:
small_db = Chroma.from_texts(texts, 
                             embedding=embedding_model)
question = "Tell me about all-white mushrooms with large fruiting bodies"
print(small_db.similarity_search(question, k=2))
print("\n\n")
print(small_db.max_marginal_relevance_search(
    question,k=3, 
    fetch_k=5))
print("\n\n")

small_db_chinese = Chroma.from_texts(texts_chinese, embedding=embedding_model)
question_chinese = "告诉我关于具有大型子实体的全白色蘑菇的信息"
print(small_db_chinese.similarity_search(question_chinese, k=2))
print("\n\n")
print(small_db_chinese.max_marginal_relevance_search(
    question_chinese,
    k=3, 
    fetch_k=5))
print("\n\n")

# 我们现在可以看到，我们检索的文档中返回了有毒的信息。

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}), Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={})]



[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}), Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}), Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={})]



[Document(page_content='一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。', metadata={}), Document(page_content='一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。', metadata={})]



[Document(page_content='一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。', metadata={}), Document(page_content='一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某

### 1.2 解决多样性：最大边际相关性(MMR)


我们刚刚通过一个示例引出了一个问题：如何加强搜索结果的多样性。
 
最大边际相关性(`Maximum marginal relevance`)试图在查询的相关性和结果的多样性之间实现两全其美。

让我们回到上一节课的一个例子，当我们通过问题对向量数据库进行相似性搜索后

我们可以看看前两个文档，只看前几个字符，可以看到它们是相同的。

注意：使用`MMR`所得出结果的差异。
当我们运行MMR后得到结果时，我们可以看到第一个与之前的相同，因为那是最相似的。
但是当我们进行到第二个时，我们可以看到它是不同的。
它在回应中获得了一些多样性。

In [24]:
question = "what did they say about matlab?"
docs_ss = vector_db.similarity_search(question,k=3)
print(docs_ss[0].page_content[:100])
print()
print(docs_ss[1].page_content[:100])
print("")

docs_mmr = vector_db.max_marginal_relevance_search(question,k=3)
print(docs_mmr[0].page_content[:100])
print("")
print(docs_mmr[1].page_content[:100])
print("\n\n")


question_chinese = "Matplotlib是什么？"
docs_ss_chinese = vector_db_chinese.similarity_search(question_chinese,k=3)
print(docs_ss_chinese[0].page_content[:100])
print("")
print(docs_ss_chinese[1].page_content[:100])
print("")

docs_mmr_chinese = vector_db_chinese.max_marginal_relevance_search(question_chinese,k=3)
print(docs_mmr_chinese[0].page_content[:100])
print("")
print(docs_mmr_chinese[1].page_content[:100])
print("\n\n")




those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 

those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 

those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 

into his office and he said, "Oh, professo r, professor, thank you so much for your 
machine learnin



第⼀回：Matplotlib 初相识
⼀、认识matplotlib
Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种

第⼀回：Matplotlib 初相识
⼀、认识matplotlib
Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种

第⼀回：Matplotlib 初相识
⼀、认识matplotlib
Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种

第⼆回：艺术画笔⻅乾坤
import numpy as np
import pandas as pd
import re
import matplotlib
import matplotlib.pyp





### 1.3 解决特殊性：使用元数据


在上一节课中，我们展示了一个问题，是询问了关于文档中某一讲的问题，但得到的结果中也包括了来自其他讲的结果。

为了解决这一问题，很多向量数据库都支持对`metadata`的操作。

`metadata`为每个嵌入的块(embedded chunk)提供上下文。

现在，我们以手动的方式来解决这个问题，我们会指定一个元数据过滤器`filter`

当然，我们不能每次都采用手动的方式来解决这个问题，这会显得不够智能

下一小节将要展示通过LLM来解决这个问题

In [32]:
question = "what did they say about regression in the third lecture?"
docs = vector_db.similarity_search(
    question,
    k=3,
    filter={"source":"docs/cs229_lectures/MachineLearning-Lecture03.pdf"}
)
# 接下来，我们可以看到结果都来自对应的章节
for d in docs:
    print(d.metadata)

question_chinese = "他们在第二讲中对Figure说了些什么？" 
docs_chinese = vector_db_chinese.similarity_search(
    question_chinese,
    k=3,
    filter={"source":"docs/matplotlib/第二回：艺术画笔见乾坤.pdf"}
)

for d in docs_chinese:
    print(d.metadata)
    

### 1.4 解决特殊性：在元数据中使用自查询检索器

我们有一个有趣的挑战：我们通常希望从查询本身来推断元数据。

为了解决这个问题，我们可以使用SelfQueryRetriever，它使用LLM来提取：
 
1. 用于向量搜索的查询(`query`)字符串，即：问题
2. 要一起传入的元数据过滤器

大多数向量数据库支持元数据过滤器，因此不需要任何新的数据库及索引。


`AttributeInfo`是我们可以指定元数据中的不同字段以及它们对应的位置。

在元数据中，我们只有两个字段，`source`和`page`。

我们将填写每个属性的名称、描述和类型的描述。

这些信息实际上将被传递给LLM，所以需要尽可能详细地描述。

In [51]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

document_content_description = "Lecture notes"
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

document_content_description_chinese = "课堂讲义"
metadata_field_info_chinese = [
    AttributeInfo(
        name="source",
        description="讲义来源于 `docs/matplotlib/第一回：Matplotlib初相识.pdf`, `docs/matplotlib/第二回：艺术画笔见乾坤.pdf`, or `docs/matplotlib/第三回：布局格式定方圆.pdf` 的其中之一",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="讲义的那一页",
        type="integer",
    ),
]


llm = OpenAI(
    # model_name = "gpt-4",
    temperature=0
    )



d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [52]:

retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_db,
    document_content_description,
    metadata_field_info,
    verbose=True
)

question = "what did they say about regression in the third lecture?"
docs = retriever.get_relevant_documents(question)

for d in docs:
    print(d.metadata)
    print(d)
print()
print()



retriever_chinese = SelfQueryRetriever.from_llm(
    llm,
    vector_db_chinese,
    document_content_description_chinese,
    metadata_field_info_chinese,
    verbose=True
)
question_chinese = "他们在第二讲中对Figure说了些什么？"  
docs_chinese = retriever_chinese.get_relevant_documents(question_chinese)
for d in docs_chinese:
    print(d.metadata)
    print(d)

query='regression' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='source', value='docs/cs229_lectures/MachineLearning-Lecture03.pdf') limit=None
{'page': 14, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
page_content='Student: It’s the lowest it –  \nInstructor (Andrew Ng) :No, exactly. Right. So zero to the same, this is not the same, \nright? And the reason is, in logi stic regression this is diffe rent from before, right? The \ndefinition of this H subscript theta of XI is not the same as the definition I was using in \nthe previous lecture. And in pa rticular this is no longer thet a transpose XI. This is not a \nlinear function anymore. This is  a logistic function of theta transpose XI. Okay? So even \nthough this looks cosmetically similar, even though this is similar on the surface, to the \nBastrian descent rule I derive d last time for least squares regression this is actually a \ntotally different learning algorithm. Okay? And it turns out th

### 1.5 其他技巧：压缩

另一种提高检索到的文档质量的方法是压缩。

与查询最相关的信息可能隐藏在具有大量不相关文本的文档中。

在应用程序中传递完整的文档可能会导致更昂贵的LLM调用和更差的响应。

上下文压缩就是为了解决这个问题。

In [37]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))
    
    
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)  # 压缩器


compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_db.as_retriever()
)
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)



compression_retriever_chinese = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_db_chinese.as_retriever()
)

question_chinese = "Matplotlib是什么？"
compressed_docs_chinese = compression_retriever_chinese.get_relevant_documents(question_chinese)
pretty_print_docs(compressed_docs_chinese)

Document 1:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 2:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 3:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's so

现在当我们提出问题后，查看结果文档

我们可以看到两件事。

1. 它们比正常文档短很多
2. 仍然有一些重复的东西，这是因为在底层我们使用的是语义搜索算法。

这就是我们在本课程前面使用MMR解决的问题。

这是一个很好的例子，你可以结合各种技术得到最好的可能结果。

## 二、结合各种技术

为了做到这一点，我们在从向量数据库创建检索器时，可以将搜索类型设置为MMR。

然后我们可以重新运行这个过程，看到我们返回的是一个过滤过的结果集，其中不包含任何重复的信息。

In [39]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_db.as_retriever(search_type = "mmr")
)

question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

compression_retriever_chinese = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_db_chinese.as_retriever(search_type = "mmr")
)

question_chinese = "Matplotlib是什么？"
compressed_docs_chinese = compression_retriever_chinese.get_relevant_documents(question_chinese)
pretty_print_docs(compressed_docs_chinese)

Document 1:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 2:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 3:

"And the student said, "Oh, it was the MATLAB." So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, and we'll actually have a short MATLAB tutorial in one o

## 三、其他类型的检索

值得注意的是，vetordb并不是唯一一种检索文档的工具。

`LangChain`检索器抽象包括其他检索文档的方式，如：`TF-IDF` 或 `SVM`。

In [40]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [41]:
# 加载PDF
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text = [p.page_content for p in pages]
joined_page_text = " ".join(all_page_text)

# 分割文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [42]:
# 加载PDF
loader_chinese = PyPDFLoader("docs/matplotlib/第一回：Matplotlib初相识.pdf")
pages_chinese = loader_chinese.load()
all_page_text_chinese = [p.page_content for p in pages_chinese]
joined_page_text_chinese = " ".join(all_page_text_chinese)

# 分割文本
text_splitter_chinese = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits_chinese = text_splitter_chinese.split_text(joined_page_text_chinese)

In [47]:
# 检索
svm_retriever = SVMRetriever.from_texts(splits, embedding_model)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

question = "What are major topics for this class?"  # 这门课的主要主题是什么？
docs_svm = svm_retriever.get_relevant_documents(question)
print(docs_svm[0])

question = "what did they say about matlab?"  # 他们关于Matlab说了些什么？
docs_tfidf = tfidf_retriever.get_relevant_documents(question)
print(docs_tfidf[0])

page_content="let me just check what questions you have righ t now. So if there are no questions, I'll just \nclose with two reminders, which are after class today or as you start to talk with other \npeople in this class, I just encourage you again to start to form project partners, to try to \nfind project partners to do your project with. And also, this is a good time to start forming \nstudy groups, so either talk to your friends  or post in the newsgroup, but we just \nencourage you to try to star t to do both of those today, okay? Form study groups, and try \nto find two other project partners.  \nSo thank you. I'm looking forward to teaching this class, and I'll see you in a couple of \ndays.   [End of Audio]  \nDuration: 69 minutes" metadata={}
page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let m

d:\Anaconda3\envs\LLM\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [49]:
# 中文版
svm_retriever_chinese = SVMRetriever.from_texts(splits_chinese, embedding_model)
tfidf_retriever_chinese = TFIDFRetriever.from_texts(splits_chinese)


question_chinese = "这门课的主要主题是什么？" 
docs_svm_chinese = svm_retriever_chinese.get_relevant_documents(question_chinese)
print(docs_svm_chinese[0])


question_chinese = "Matplotlib是什么？"
docs_tfidf_chinese = tfidf_retriever_chinese.get_relevant_documents(question_chinese)
print(docs_tfidf_chinese[0])

page_content='fig, ax = plt.subplots()  \n# step4 绘制图像，  这⼀模块的扩展参考第⼆章进⼀步学习\nax.plot(x, y, label=\'linear\')  \n# step5 添加标签，⽂字和图例，这⼀模块的扩展参考第四章进⼀步学习\nax.set_xlabel(\'x label\') \nax.set_ylabel(\'y label\') \nax.set_title("Simple Plot")  \nax.legend() ;\n思考题\n请思考两种绘图模式的优缺点和各⾃适合的使⽤场景\n在第五节绘图模板中我们是以 OO 模式作为例⼦展示的，请思考并写⼀个 pyplot 绘图模式的简单模板' metadata={}
page_content='fig, ax = plt.subplots()  \n# step4 绘制图像，  这⼀模块的扩展参考第⼆章进⼀步学习\nax.plot(x, y, label=\'linear\')  \n# step5 添加标签，⽂字和图例，这⼀模块的扩展参考第四章进⼀步学习\nax.set_xlabel(\'x label\') \nax.set_ylabel(\'y label\') \nax.set_title("Simple Plot")  \nax.legend() ;\n思考题\n请思考两种绘图模式的优缺点和各⾃适合的使⽤场景\n在第五节绘图模板中我们是以 OO 模式作为例⼦展示的，请思考并写⼀个 pyplot 绘图模式的简单模板' metadata={}


d:\Anaconda3\envs\LLM\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
